In [35]:
import numpy as np
import pandas as pd

In [36]:
data = pd.read_csv("./data/Gemstone.csv")

In [37]:
data = data.drop(columns="id")

In [38]:
x = data.drop(columns="price")

In [39]:
y=data[["price"]]

In [40]:
categorical_col = x.select_dtypes(include="object").columns

In [41]:
numerical_col = x.select_dtypes(exclude="object").columns

In [42]:
# Define the rankings according to preference 

In [43]:
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [44]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [45]:
num_pipeline=Pipeline(
    
    steps=[
        
        ('imputer',SimpleImputer()),
        ('scaler',StandardScaler())
    ]
)

In [46]:
cat_pipeline=Pipeline(
    
    steps=[
        
        ('imputer',SimpleImputer(strategy="most_frequent")),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
    ]
)

In [47]:
preprocessor = ColumnTransformer(
    
    [
        
        ('num_pipeline',num_pipeline,numerical_col),
        ('cat_pipeline',cat_pipeline,categorical_col)
    ]
)

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=42)

In [50]:
x_train

,carat,cut,color,clarity,depth,table,x,y,z
11504,0.41,Ideal,E,VVS2,60.6,56.0,4.85,4.80,2.93
95284,1.23,Very Good,H,VS1,59.9,59.0,6.91,7.01,4.19
184777,1.70,Premium,H,VS2,62.0,58.0,7.61,7.66,4.74
5419,0.33,Ideal,F,VVS1,61.2,56.0,4.47,4.44,2.73
45466,0.33,Very Good,I,SI1,62.1,58.0,4.41,4.45,2.75
...,...,...,...,...,...,...,...,...,...
119879,0.50,Very Good,E,SI1,60.2,61.0,5.11,5.15,3.09
103694,1.91,Very Good,F,SI1,62.3,62.0,7.85,7.79,4.87
131932,1.22,Premium,G,VS2,62.8,58.0,6.82,6.74,4.26
146867,0.31,Very Good,G,VVS1,61.1,56.0,4.37,4.40,2.67


In [51]:
preprocessor.fit_transform(x_train)

array([[-0.82314374, -1.12998781, -0.64189666, ...,  4.        ,
         1.        ,  5.        ],
       [ 0.94502267, -1.77782269,  0.92190185, ...,  2.        ,
         4.        ,  4.        ],
       [ 1.9584839 ,  0.16568195,  0.40063568, ...,  3.        ,
         4.        ,  3.        ],
       ...,
       [ 0.92345966,  0.90606467,  0.40063568, ...,  3.        ,
         3.        ,  3.        ],
       [-1.03877378, -0.66724861, -0.64189666, ...,  2.        ,
         3.        ,  6.        ],
       [-1.03877378, -0.01941373,  0.92190185, ...,  2.        ,
         3.        ,  1.        ]])

In [52]:
preprocessor.transform(x_test)

array([[-0.62907669,  0.25822979, -0.12063049, ...,  4.        ,
         0.        ,  2.        ],
       [ 2.60537405, -2.14801405, -0.12063049, ...,  2.        ,
         3.        ,  1.        ],
       [-1.1250258 , -1.22253565,  0.92190185, ...,  3.        ,
         1.        ,  6.        ],
       ...,
       [-0.82314374, -0.01941373, -0.64189666, ...,  4.        ,
         3.        ,  6.        ],
       [ 0.90189666, -0.66724861,  1.44316802, ...,  3.        ,
         5.        ,  2.        ],
       [ 0.47063656,  0.90606467, -0.64189666, ...,  2.        ,
         1.        ,  3.        ]])

In [53]:
preprocessor.get_feature_names_out()

array(['num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [56]:
x_train = pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())

In [59]:
x_test = pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [62]:
# Model building

In [63]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [64]:
models = {
    'LinearRegression':LinearRegression(),
    'Ridge':Ridge(),
    'Lasso':Lasso(),
    'ElasticNet':ElasticNet()
}

In [78]:
def evaluate_model(true,predicted):
    r2_square = r2_score(true,predicted)
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    
    return r2_square,mae,mse

In [80]:
for i in range(len(models)):
    model=list(models.values())[i]
    model.fit(x_train,y_train)
    
    y_pred = model.predict(x_test)
    
    r2_square ,mae ,mse = evaluate_model(y_test,y_pred)
    
    print(list(models.keys())[i])
    print("Model training performance : ")
    print("MSE : ",mse)
    print("MAE",mae)
    print("R2_score",r2_square)
    
    print("="*35)
    print("\n")
    
    

LinearRegression
Model training performance : 
MSE :  1029473.3531156847
MAE 675.0758270067478
R2_score 0.9362906819996047


Ridge
Model training performance : 
MSE :  1029482.832981572
MAE 675.1087261318746
R2_score 0.9362900953347935


Lasso
Model training performance : 
MSE :  1029534.1175907081
MAE 676.3788803086726
R2_score 0.9362869215688461


ElasticNet
Model training performance : 
MSE :  2291130.057094769
MAE 1051.1722281863017
R2_score 0.8582126162411592


